# 

In [1]:
import numpy as np
import pandas as pd



This Notebook demonstrates how to use our big dataset class

-----------------------------------------------------------------------------------------------------------

In [2]:
#Final big-ass table after the join....

In [31]:
from MovieLens import Movie_100K

In [32]:
dataset = Movie_100K()

In [33]:
dataset[1000:1003]

{'title': {1000: 'Dead Man Walking (1995)',
  1001: 'Seven (Se7en) (1995)',
  1002: 'Usual Suspects, The (1995)'},
 'date': {1000: array([1.        , 0.5       , 0.8660254 , 0.5       , 0.97952994]),
  1001: array([1.        , 0.5       , 0.8660254 , 0.5       , 0.97952994]),
  1002: array([ 1.        , -0.8660254 , -0.5       ,  0.8660254 , -0.95413926])},
 'genre': {1000: '[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]',
  1001: '[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0]',
  1002: '[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0]'},
 'user_id': {1000: 109, 1001: 109, 1002: 109},
 'item_id': {1000: 9, 1001: 11, 1002: 12},
 'rating': {1000: 3, 1001: 4, 1002: 4},
 'timestamp': {1000: 880564607, 1001: 880572786, 1002: 880577542},
 'age': {1000: 29, 1001: 29, 1002: 29},
 'gender': {1000: 1, 1001: 1, 1002: 1},
 'occupation': {1000: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]),
  1001: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [34]:
All_DataFrame = pd.DataFrame(dataset[:])

In [35]:
# work with the title
Title_frame = pd.DataFrame(All_DataFrame['title'])
def trim(row):
    row['title'] = row['title'][0:len(row)-7]#excluding the year in the title
    return row
Title_frame = Title_frame.apply(trim, axis = 1)

In [36]:
Title_frame

,title
0,Toy Story
1,Get Shorty
2,Copycat
3,Twelve Monkeys
4,Babe
...,...
99995,"Saint, The"
99996,Tomorrow Never Dies
99997,"Peacemaker, The"
99998,Home Alone 3


In [37]:
# stopwords + encode.
import spacy
NLP = spacy.load("en_core_web_sm")
def lemmatize_and_stop(row):
    
    doc = NLP(row['title'])
    lemmatized_words = [token.lemma_ for token in doc]
    row['title'] = ' '.join(lemmatized_words)
    return row

    

In [38]:
Title_frame = Title_frame.apply(lemmatize_and_stop, axis = 1)


In [39]:
import re

In [40]:
Title_frame

,title
0,Toy Story
1,get Shorty
2,copycat
3,twelve monkey
4,babe
...,...
99995,"Saint , the"
99996,tomorrow never die
99997,"Peacemaker , the"
99998,home alone 3


In [45]:
def Format_lower(row):
    #print(row[1])
    string = re.sub("r'[^a-zA-Z \s]+$", '', row['title']).lower() #re.sub(r'[^a-zA-Z\s]', '', text)
    #string = re.sub(r'[,\.\:\(\)\;\!\/\-]', '', row['title'])
    #string = re.sub(r'[&]', 'and', string )
    row['title'] = string
    return row
    

In [46]:
Title_frame2 = Title_frame2.apply(Format_lower, axis = 1)

In [47]:
Title_frame2

,title
0,toy story
1,get shorty
2,copycat
3,twelve monkey
4,babe
...,...
99995,saint the
99996,tomorrow never die
99997,peacemaker the
99998,home alone 3


In [48]:
Title_frame2['title'].to_csv('/home/yiyangl/PY_PROJECTS/Datasets/ml-100k/processed_title.csv')

Potentially useful variables by intuition:

Scheme 1
*Action space*
Each movie vector/id, which should include/be linked to：
1. The titles of the movies: the similarity can be used as a criteria for recommendation
2. Year, title, genre, and tags, and average rating of the movie can help.

*Observations*
1. An user's average rating (related to the genre or not)
2. Some generization of the users' preference. (year, title, tags, average rating)

Reward = the actual rating of the user.
   

In [2]:
# TO_INPUTID. TO_WORD_EMBEDDING

In [4]:
processed_text = list(pd.read_csv('/home/yiyangl/PY_PROJECTS/Datasets/ml-100k/processed_title.csv')['title'])

In [5]:
from transformers import AutoTokenizer, MobileBertForMaskedLM
import torch

In [6]:
processed_text = [text if text else 'None' for text in processed_text]

In [7]:
processed_text[:10]

['toy story',
 'get shorty',
 'copycat',
 'twelve monkey',
 'babe',
 'dead man walk',
 'seven  se7en ',
 'usual suspects  the',
 "mr holland 's opus",
 'from dusk till dawn']

In [8]:
Tokenizer = AutoTokenizer.from_pretrained('google/mobilebert-uncased')
text_batch = [Tokenizer.encode_plus(
            text,
            add_special_tokens=True,  # Add [CLS] and [SEP]
            max_length=10,
            padding='max_length',
            truncation=True,  # Truncate to max_length
            return_attention_mask=True,  # Return attention masks
            return_tensors='pt'  # Return PyTorch tensors
            ) if text is not None else None for text in processed_text]

In [9]:
id_list = [x['input_ids'].squeeze() for x in text_batch]
att_list = [x['attention_mask'].squeeze() for x in text_batch]
input_ids = torch.stack(id_list, dim = 0)
attention_mask = torch.stack(att_list, dim = 0)

In [10]:
torch.save(input_ids,'/home/yiyangl/PY_PROJECTS/Datasets/ml-100k/input_ids.pt')
torch.save(attention_mask,'/home/yiyangl/PY_PROJECTS/Datasets/ml-100k/att.pt')

In [11]:
import torch
from transformers import MobileBertModel

In [12]:
text_encoder = MobileBertModel.from_pretrained('google/mobilebert-uncased')

In [13]:
batchsize = 1
textemblist = []
for i in range(90000,90000+int(10000/batchsize)):
    out =text_encoder(input_ids=input_ids[i*batchsize:(i+1)*batchsize,:], attention_mask=attention_mask[i*batchsize:(i+1)*batchsize,:])
    out = out.last_hidden_state
    textemblist.append(out[:,0,:])
    if i%5000 ==4999:
        encoded_text = torch.stack(textemblist, dim=0)
        torch.save(encoded_text.detach(), '/home/yiyangl/PY_PROJECTS/Datasets/ml-100k/encoded_text'+str(i+1)+'.pt')

In [14]:
torch_list = []
for i in range(10):
    
    filename =  '/home/yiyangl/PY_PROJECTS/Datasets/ml-100k/encoded_text'+str(i+1)+'0000.pt'
    temp = torch.load(filename)
    torch_list.append(temp)

In [16]:
encoded_text = torch.stack(torch_list, dim=0)
torch.save(encoded_text, '/home/yiyangl/PY_PROJECTS/Datasets/ml-100k/encoded_text_all.pt')

In [17]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
encoded_text.shape

----------------------------------First explore the possible genres---------------------------------------

In [21]:
one_to = torch.load( '/home/yiyangl/PY_PROJECTS/Datasets/ml-100k/encoded_text20000.pt')

In [22]:
one_to.shape

torch.Size([10000, 1, 512])

In [19]:
torch.save(one_to[:10000,:,:].detach() , '/home/yiyangl/PY_PROJECTS/Datasets/ml-100k/encoded_text10000.pt' )

-----------------------------Next: the tags------------------------------
What someone is saying about the film. Can be understood as one sentense film criticism
Can be used for 1. sentiment analysis (may overlap with rating) 2. keyword extraction.

hat we potentially need in our model structure: 
something that can relate rating to subcategories
something that can relate user to keywords and subcategories
